In [ ]:
import numpy as np
from scipy.io import loadmat, savemat
from sklearn.metrics import classification_report, coverage_error, label_ranking_average_precision_score, hamming_loss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.calibration import CalibratedClassifierCV
import os
import time

def main_rf(scenario_name, snr_value, top_k=44):
    # Load data
    train_X = loadmat(f'/path/{scenario_name}-{snr_value}-train/data.mat')['datat']
    train_Y = loadmat(f'/path/{scenario_name}-{snr_value}-train-labels/labels.mat')['Labels']
    valid_X = loadmat(f'/path/{scenario_name}-{snr_value}-valid/data.mat')['datat']
    valid_Y = loadmat(f'/path/{scenario_name}-{snr_value}-valid-labels/labels.mat')['Labels']
    test_X = loadmat(f'/path/{scenario_name}-{snr_value}-test/data.mat')['datat']
    test_Y = loadmat(f'/path/{scenario_name}-{snr_value}-test-labels/labels.mat')['Labels']
    
    # Reshape data
    train_X = np.transpose(train_X, (3, 0, 1, 2)).reshape(6000, -1)
    valid_X = np.transpose(valid_X, (3, 0, 1, 2)).reshape(2000, -1)
    test_X = np.transpose(test_X, (3, 0, 1, 2)).reshape(2000, -1)

    # Concatenate train and validation data
    train_X = np.concatenate((train_X, valid_X))
    train_Y = np.concatenate((train_Y, valid_Y))

    # Standardize the data
    scaler = StandardScaler()
    train_X = scaler.fit_transform(train_X)
    test_X = scaler.transform(test_X)

    # Reduce dimensionality with PCA
    pca = PCA(n_components=10)
    train_X = pca.fit_transform(train_X)
    test_X = pca.transform(test_X)

    # Initialize RandomForestClassifier
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    # Initialize CalibratedClassifierCV with 'sigmoid' calibration
    calibrated_rf = CalibratedClassifierCV(rf, method='sigmoid')

    # Initialize ClassifierChain with CalibratedClassifierCV
    chain_classifier_rf = ClassifierChain(calibrated_rf, order='random', random_state=42)

    # Fit the data to the ClassifierChain with the calibrated RandomForestClassifier
    chain_classifier_rf.fit(train_X, train_Y)

    start_inference_time = time.time()

    # Predict probabilities for the test set using the chain classifier
    y_pred_prob_rf = np.array(chain_classifier_rf.predict_proba(test_X))
    print(y_pred_prob_rf)
    sorted_indices_rf = np.argsort(-y_pred_prob_rf, axis=1)

    # Initialize array to store top k labels for each sample
    y_pred_chain_rf = np.zeros_like(test_Y)
    
    # Select top k labels for each sample
    for i in range(len(test_Y)):
        top_indices = sorted_indices_rf[i, :top_k]
        y_pred_chain_rf[i, top_indices] = 1

    end_inference_time = time.time()
    inference_time = end_inference_time - start_inference_time
    # Print the inference time
    print(f"Inference Time: {inference_time} seconds", flush=True)

    # Compute classification metrics
    report_rf = classification_report(test_Y, y_pred_chain_rf)
    coverage_err_rf = coverage_error(test_Y, y_pred_prob_rf)
    avg_precision_rf = label_ranking_average_precision_score(test_Y, y_pred_prob_rf)
    hamming_rf = hamming_loss(test_Y, y_pred_chain_rf)

    print("Classification Report (RandomForestClassifier and CalibratedClassifierCV):\n", report_rf)
    print(f"Coverage Error (RandomForestClassifier and CalibratedClassifierCV): {coverage_err_rf:.4f}")
    print(f"Label Ranking Average Precision Score (RandomForestClassifier and CalibratedClassifierCV): {avg_precision_rf:.4f}")
    print(f"Hamming Loss (RandomForestClassifier and CalibratedClassifierCV): {hamming_rf:.4f}")

    # Save predictions
    directory_rf = "/your-path/"
    os.makedirs(directory_rf, exist_ok=True)
    filename_rf = f"{scenario_name}_{snr_value}_RF_Chain.mat"
    filepath_rf = os.path.join(directory_rf, filename_rf)

    try:
        savemat(filepath_rf, {"y_pred_chain_rf": y_pred_chain_rf})
    except Exception as e:
        print("Error saving file (RandomForestClassifier and CalibratedClassifierCV):", e)

# Example usage:
if __name__ == "__main__":
    scenario_name = "44"
    snr_value = "snr10"
    main_rf(scenario_name, snr_value)